In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .config("spark.driver.extraJavaOptions", "-Dlog4j.configuration=file:log4j.properties") \
    .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

2026-02-09T09:25:54.374696403Z main ERROR Reconfiguration failed: No configuration found for '4e0e2f2a' at 'null' in 'null'
2026-02-09T09:25:54.614696636Z main ERROR Reconfiguration failed: No configuration found for 'Default' at 'null' in 'null'


Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
26/02/09 09:25:54 WARN Utils: Your hostname, codespaces-8b0c36, resolves to a loopback address: 127.0.0.1; using 10.0.11.36 instead (on interface eth0)
26/02/09 09:25:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/02/09 09:25:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/02/09 09:25:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
26/02/09 09:25:57 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')
df_green.registerTempTable('green')

/home/codespace/.python/current/lib/python3.12/site-packages/pyspark/sql/classic/dataframe.py:178: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [7]:
df_green_revenue = spark.sql("""
SELECT 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    ROUND(SUM(total_amount), 2) AS amount,
    COUNT(1) AS number_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [8]:
df_green_revenue.show()

[Stage 6:>                                                          (0 + 2) / 2]

+-------------------+----+-------+--------------+
|               hour|zone| amount|number_records|
+-------------------+----+-------+--------------+
|2020-01-01 00:00:00|   7| 769.73|            45|
|2020-01-01 00:00:00|  17| 195.03|             9|
|2020-01-01 00:00:00|  18|    7.8|             1|
|2020-01-01 00:00:00|  22|   15.8|             1|
|2020-01-01 00:00:00|  24|   87.6|             3|
|2020-01-01 00:00:00|  25|  531.0|            26|
|2020-01-01 00:00:00|  29|   61.3|             1|
|2020-01-01 00:00:00|  32|  68.95|             2|
|2020-01-01 00:00:00|  33| 317.27|            11|
|2020-01-01 00:00:00|  35| 129.96|             5|
|2020-01-01 00:00:00|  36| 295.34|            11|
|2020-01-01 00:00:00|  37| 175.67|             6|
|2020-01-01 00:00:00|  38|  98.79|             2|
|2020-01-01 00:00:00|  40| 168.98|             8|
|2020-01-01 00:00:00|  41|1363.96|            84|
|2020-01-01 00:00:00|  42| 799.76|            52|
|2020-01-01 00:00:00|  43| 107.52|             6|


In [9]:
df_green_revenue \
    .repartition(20, "hour", "zone") \
    .write.parquet('data/report/revenue/green', mode='overwrite')

In [10]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable('yellow')

In [11]:
df_yellow_revenue = spark.sql("""
SELECT 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    ROUND(SUM(total_amount), 2) AS amount,
    COUNT(1) AS number_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [12]:
df_yellow_revenue \
    .repartition(20, "hour", "zone") \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

In [13]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [14]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [18]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer').fillna(0)

In [19]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

In [20]:
df_join.show()

[Stage 41:>                                                         (0 + 1) / 1]

+-------------------+----+------------+--------------------+-------------+---------------------+
|               hour|zone|green_amount|green_number_records|yellow_amount|yellow_number_records|
+-------------------+----+------------+--------------------+-------------+---------------------+
|2020-01-01 00:00:00|   3|         0.0|                   0|         25.0|                    1|
|2020-01-01 00:00:00|   4|         0.0|                   0|       1004.3|                   57|
|2020-01-01 00:00:00|   7|      769.73|                  45|       455.17|                   38|
|2020-01-01 00:00:00|  12|         0.0|                   0|        107.0|                    6|
|2020-01-01 00:00:00|  13|         0.0|                   0|       1214.8|                   56|
|2020-01-01 00:00:00|  18|         7.8|                   1|          5.8|                    1|
|2020-01-01 00:00:00|  29|        61.3|                   1|          0.0|                    0|
|2020-01-01 00:00:00|  36|    

In [21]:
df_join = spark.read.parquet('data/report/revenue/total')
df_zones = spark.read.parquet('zones/')

In [25]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID).drop('LocationID', 'zone')
df_result.show()
df_result.write.mode("overwrite").parquet('tmp/revenue-zones')

+-------------------+------------+--------------------+-------------+---------------------+---------+------------+
|               hour|green_amount|green_number_records|yellow_amount|yellow_number_records|  Borough|service_zone|
+-------------------+------------+--------------------+-------------+---------------------+---------+------------+
|2020-01-01 00:00:00|         0.0|                   0|        42.41|                    2|   Queens|   Boro Zone|
|2020-01-01 00:00:00|         0.0|                   0|          8.8|                    1| Brooklyn|   Boro Zone|
|2020-01-01 00:00:00|         0.0|                   0|        34.09|                    1|   Queens|   Boro Zone|
|2020-01-01 00:00:00|      195.03|                   9|       220.21|                    8| Brooklyn|   Boro Zone|
|2020-01-01 00:00:00|        15.8|                   1|          0.0|                    0| Brooklyn|   Boro Zone|
|2020-01-01 00:00:00|        87.6|                   3|       754.95|           